In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
groups = ['Queen', 'Led Zeppelin', 'Black Sabbath', 'Deep Purple', 'Scorpions',
          'Metallica','Iron Maiden',  'Megadeth', 'Judas Priest', 'Motorhead',
          'Kendrick Lamar', 'Tyler, the Creator', 'Kanye West', '2pac', 'Jay-Z',
          'Radiohead', 'Bjork', 'My Bloody Valentine', 'Madvillain', 'Godspeed You Black Emperor!',
          'Michael Jackson', 'Madonna', 'George Michael', 'Janet Jackson', 'Prince']
group_df = pd.DataFrame(index=groups, columns=groups)
popularities = []
for i in range(len(groups) // 5):
    popularities.extend(range(5, 0, -1))


for i in range(len(groups) // 5):
    for j in range(5 * i):
        for k in range(5 * i, 5 * i + 5):
            group_df.iloc[j, k] = group_df.iloc[k, j] = 1
    for j in range(5 * i, 5 * i + 5):
        for k in range(5 * i, 5 * i + 5):
            if j != k:
                group_df.iloc[j, k] = group_df.iloc[k, j] = 3
            else:
                group_df.iloc[j, k] = 0
    for j in range(5 * i + 5, len(groups)):
        for k in range(5 * i, 5 * i + 5):
            group_df.iloc[j, k] = group_df.iloc[k, j] = 3

for i in range(len(groups)):
    group_df.iloc[i] *= popularities[i]
group_df

,Queen,Led Zeppelin,Black Sabbath,Deep Purple,Scorpions,Metallica,Iron Maiden,Megadeth,Judas Priest,Motorhead,...,Radiohead,Bjork,My Bloody Valentine,Madvillain,Godspeed You Black Emperor!,Michael Jackson,Madonna,George Michael,Janet Jackson,Prince
Queen,0,15,15,15,15,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
Led Zeppelin,12,0,12,12,12,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
Black Sabbath,9,9,0,9,9,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
Deep Purple,6,6,6,0,6,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
Scorpions,3,3,3,3,0,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Metallica,5,5,5,5,5,0,15,15,15,15,...,5,5,5,5,5,5,5,5,5,5
Iron Maiden,4,4,4,4,4,12,0,12,12,12,...,4,4,4,4,4,4,4,4,4,4
Megadeth,3,3,3,3,3,9,9,0,9,9,...,3,3,3,3,3,3,3,3,3,3
Judas Priest,2,2,2,2,2,6,6,6,0,6,...,2,2,2,2,2,2,2,2,2,2
Motorhead,1,1,1,1,1,3,3,3,3,0,...,1,1,1,1,1,1,1,1,1,1


In [3]:
transaction_count = 2000
transactions = {'Transaction': [], 'Item': []}
for tr_id in range(transaction_count):
    items = random.choices(groups, weights=popularities)
    item_count = random.choices(range(8), [1, 7, 10, 6, 5, 3, 2, 1])[0]
    items.extend(random.choices(groups, weights=group_df[items[0]], k=item_count - 1))
    for item in items:
        transactions['Transaction'].append(tr_id + 1)
        transactions['Item'].append(item)
transactions = pd.DataFrame(transactions)
transactions

,Transaction,Item
0,1,Bjork
1,1,Black Sabbath
2,1,My Bloody Valentine
3,2,Michael Jackson
4,2,Queen
...,...,...
5765,1998,Bjork
5766,1998,Michael Jackson
5767,1999,Madvillain
5768,1999,Iron Maiden


In [4]:
transactions.to_csv('groups.csv', index=False, encoding='utf-8')

# Анализ датасета

In [5]:
import pandas as pd
from collections import defaultdict
import datetime as dt

In [6]:
df = pd.read_csv("./groups.csv", encoding='utf-8')

In [7]:
df.head(100)

,Transaction,Item
0,1,Bjork
1,1,Black Sabbath
2,1,My Bloody Valentine
3,2,Michael Jackson
4,2,Queen
...,...,...
95,33,Iron Maiden
96,34,George Michael
97,34,Michael Jackson
98,35,Led Zeppelin


In [53]:
df['Date'] = df['Date'].apply(lambda d: dt.datetime.strptime(d, '%Y-%m-%d'))

In [54]:
df = df[df['Date'] <= dt.datetime(2016, 10, 30) + dt.timedelta(days=1)]

In [8]:
df

,Transaction,Item
0,1,Bjork
1,1,Black Sabbath
2,1,My Bloody Valentine
3,2,Michael Jackson
4,2,Queen
...,...,...
5765,1998,Bjork
5766,1998,Michael Jackson
5767,1999,Madvillain
5768,1999,Iron Maiden


In [9]:
df['Item'] = df['Item'].apply(lambda x: x.lower().strip())
df['Item'] = df.loc[df['Item'] != 'none', ['Item']]
df = df.dropna()

In [10]:
len(df['Item'].unique())

25

In [11]:
df['Transaction'].max() == len(df['Transaction'].unique())

np.True_

In [12]:
unique_items = df['Item'].unique()
_dict = defaultdict(list)
for transaction_number in df['Transaction'].unique():
    products = df.loc[df['Transaction'] == transaction_number]['Item'].to_list()
    for pr in unique_items:
        if pr in products:
            _dict[pr].append(1)
        else:
            _dict[pr].append(0)
norm_df = pd.DataFrame(_dict)
norm_df

,bjork,black sabbath,my bloody valentine,michael jackson,queen,janet jackson,"tyler, the creator",led zeppelin,metallica,deep purple,...,jay-z,2pac,george michael,kanye west,radiohead,prince,judas priest,motorhead,madvillain,godspeed you black emperor!
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
1997,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [13]:
norm_df.sum()

bjork                          308
black sabbath                  213
my bloody valentine            249
michael jackson                346
queen                          341
janet jackson                  155
tyler, the creator             281
led zeppelin                   274
metallica                      331
deep purple                    133
kendrick lamar                 353
madonna                        281
iron maiden                    300
scorpions                       94
megadeth                       227
jay-z                           88
2pac                           129
george michael                 229
kanye west                     212
radiohead                      356
prince                          72
judas priest                   180
motorhead                       73
madvillain                     162
godspeed you black emperor!     94
dtype: int64

# Алгоритм Apriori

In [14]:
from itertools import combinations, permutations
from tqdm import tqdm

In [15]:
def count_rows(items: list[str]):
    condition = (norm_df[items[0]] == 1)
    for i in range(1, len(items)):
        condition &= (norm_df[items[i]] == 1)
    cnt = len(norm_df[condition])
    return cnt

In [16]:
k = 1
f = 2
results = []
while True:
    f_k = []
    if k == 1:
        for item in unique_items:
            if norm_df.sum()[item] >= f:
                f_k.append(item)
    else:
        if k == 2:
            combs = [x for x in combinations(results[0], r=2)]
        else:
            previous = results[k - 2]
            possible = []
            for i, elem in enumerate(previous):
                elem = sorted(elem)
                _set = set(elem[:-1])
                if any(_set.issubset(set(x)) for x in possible):
                    continue
                for j, elem1 in enumerate(previous):
                    if i == j:
                        continue
                    if _set.issubset(set(elem1)):
                        possible.append(tuple(set(elem).union(set(elem1))))
            
            combs = []
            for elem in possible:
                flag = True
                for subset in combinations(elem, r=k-1):
                    cnt = count_rows(subset)
                    if cnt < f:
                        flag = False
                        break
                if flag:
                    combs.append(elem)
        
        for elem in tqdm(combs):
            cnt = count_rows(elem)
            if cnt >= f:
                f_k.append(elem)
    
    if len(f_k) == 0:
        break
    results.append(f_k)
    k += 1
print(results[1:])

100%|██████████| 13/13 [00:00<00:00, 1319.54it/s]
0it [00:00, ?it/s]

[[('bjork', 'black sabbath'), ('bjork', 'my bloody valentine'), ('bjork', 'michael jackson'), ('bjork', 'queen'), ('bjork', 'janet jackson'), ('bjork', 'tyler, the creator'), ('bjork', 'led zeppelin'), ('bjork', 'metallica'), ('bjork', 'deep purple'), ('bjork', 'kendrick lamar'), ('bjork', 'madonna'), ('bjork', 'iron maiden'), ('bjork', 'scorpions'), ('bjork', 'megadeth'), ('bjork', 'jay-z'), ('bjork', '2pac'), ('bjork', 'george michael'), ('bjork', 'kanye west'), ('bjork', 'radiohead'), ('bjork', 'prince'), ('bjork', 'judas priest'), ('bjork', 'motorhead'), ('bjork', 'madvillain'), ('bjork', 'godspeed you black emperor!'), ('black sabbath', 'my bloody valentine'), ('black sabbath', 'michael jackson'), ('black sabbath', 'queen'), ('black sabbath', 'janet jackson'), ('black sabbath', 'tyler, the creator'), ('black sabbath', 'led zeppelin'), ('black sabbath', 'metallica'), ('black sabbath', 'deep purple'), ('black sabbath', 'kendrick lamar'), ('black sabbath', 'madonna'), ('black sabbath

In [17]:
results.pop(0)
while type(results[0]) != tuple:
    for elem in results[0]:
        results.append(elem)
    results.pop(0)
results

[('bjork', 'black sabbath'),
 ('bjork', 'my bloody valentine'),
 ('bjork', 'michael jackson'),
 ('bjork', 'queen'),
 ('bjork', 'janet jackson'),
 ('bjork', 'tyler, the creator'),
 ('bjork', 'led zeppelin'),
 ('bjork', 'metallica'),
 ('bjork', 'deep purple'),
 ('bjork', 'kendrick lamar'),
 ('bjork', 'madonna'),
 ('bjork', 'iron maiden'),
 ('bjork', 'scorpions'),
 ('bjork', 'megadeth'),
 ('bjork', 'jay-z'),
 ('bjork', '2pac'),
 ('bjork', 'george michael'),
 ('bjork', 'kanye west'),
 ('bjork', 'radiohead'),
 ('bjork', 'prince'),
 ('bjork', 'judas priest'),
 ('bjork', 'motorhead'),
 ('bjork', 'madvillain'),
 ('bjork', 'godspeed you black emperor!'),
 ('black sabbath', 'my bloody valentine'),
 ('black sabbath', 'michael jackson'),
 ('black sabbath', 'queen'),
 ('black sabbath', 'janet jackson'),
 ('black sabbath', 'tyler, the creator'),
 ('black sabbath', 'led zeppelin'),
 ('black sabbath', 'metallica'),
 ('black sabbath', 'deep purple'),
 ('black sabbath', 'kendrick lamar'),
 ('black sabba

In [18]:
rules = pd.DataFrame(columns=['Условие', 'Следствие', 'Поддержка', 'Достоверность'])
# rules = {'Условие': [], 'Следствие': [], 'Поддержка': [], 'Достоверность': []}
for elem in results:
    for p in combinations(elem, r=len(elem)-1):
        condition = list(p)
        consequence = list(set(elem).difference(set(p)))
        support = count_rows(condition + consequence) / len(norm_df) * 100
        credibility = count_rows(condition + consequence) / count_rows(condition) * 100
        rules.loc[len(rules)] = [condition, consequence, support, credibility]
rules

,Условие,Следствие,Поддержка,Достоверность
0,[bjork],[black sabbath],1.55,10.064935
1,[black sabbath],[bjork],1.55,14.553991
2,[bjork],[my bloody valentine],2.50,16.233766
3,[my bloody valentine],[bjork],2.50,20.080321
4,[bjork],[michael jackson],2.50,16.233766
...,...,...,...,...
640,"[black sabbath, judas priest]",[bjork],0.10,8.000000
641,"[black sabbath, bjork, radiohead]",[my bloody valentine],0.10,40.000000
642,"[black sabbath, bjork, my bloody valentine]",[radiohead],0.10,50.000000
643,"[black sabbath, radiohead, my bloody valentine]",[bjork],0.10,22.222222


In [19]:
rules['Поддержка'].max()

np.float64(4.9)

In [20]:
rules['Достоверность'].max()

np.float64(50.0)

In [21]:
rules[(rules['Достоверность'] >= 50)]

,Условие,Следствие,Поддержка,Достоверность
642,"[black sabbath, bjork, my bloody valentine]",[radiohead],0.1,50.0


# Алгоритм Eclat

In [22]:
eclat_dict = {'Item': [], 'Transactions': []}
min_support = 4
for item in sorted(unique_items):
    transactions = norm_df[norm_df[item] == 1].index.to_list()
    eclat_dict['Item'].append([item])
    eclat_dict['Transactions'].append(transactions)
eclat_df = pd.DataFrame(eclat_dict)
eclat_df = eclat_df[eclat_df['Transactions'].apply(lambda t: len(t) >= min_support)]
eclat_df

,Item,Transactions
0,[2pac],"[9, 14, 23, 35, 48, 57, 69, 70, 77, 83, 89, 95..."
1,[bjork],"[0, 6, 15, 18, 28, 39, 45, 52, 55, 58, 78, 79,..."
2,[black sabbath],"[0, 1, 2, 13, 16, 17, 27, 30, 37, 41, 47, 51, ..."
3,[deep purple],"[3, 7, 17, 22, 40, 45, 59, 65, 85, 97, 103, 11..."
4,[george michael],"[13, 14, 26, 33, 42, 43, 45, 56, 62, 65, 83, 9..."
5,[godspeed you black emperor!],"[45, 83, 89, 151, 172, 216, 234, 240, 247, 254..."
6,[iron maiden],"[5, 20, 27, 32, 36, 39, 40, 47, 61, 62, 66, 67..."
7,[janet jackson],"[1, 13, 40, 51, 54, 94, 137, 148, 160, 190, 20..."
8,[jay-z],"[8, 53, 76, 78, 88, 102, 131, 178, 196, 209, 2..."
9,[judas priest],"[16, 17, 30, 36, 41, 45, 54, 62, 70, 95, 102, ..."


In [23]:
min_support = 4
start_point = 0
k = 2
while True:
    end_point = len(eclat_df)
    
    new_dict = {'Item': [], 'Transactions': []}
    for i in tqdm(range(start_point, end_point)):
        for j in range(i + 1, end_point):
            item1, item2 = eclat_df.iloc[i]['Item'], eclat_df.iloc[j]['Item']
            if k > 2 and all(x == y for x in item1[:k - 2] for y in item2[:k - 2]):
                break

            tr1, tr2 = eclat_df.iloc[i]['Transactions'], eclat_df.iloc[j]['Transactions']
            if len(set(item1).union(set(item2))) == k:
                new_dict['Item'].append(sorted(set(item1).union(set(item2))))
                new_dict['Transactions'].append(list(set(tr1).intersection(set(tr2))))
    
    eclat_df = pd.concat([
        eclat_df,
        pd.DataFrame(new_dict)
    ], ignore_index=True)
    if len(eclat_df) == end_point:
        break
    k += 1
    start_point = end_point
    eclat_df = eclat_df[eclat_df['Transactions'].apply(lambda t: len(t) >= min_support)]

eclat_df

100%|██████████| 6/6 [00:00<00:00, 4390.41it/s]
0it [00:00, ?it/s]


,Item,Transactions
0,[2pac],"[9, 14, 23, 35, 48, 57, 69, 70, 77, 83, 89, 95..."
1,[bjork],"[0, 6, 15, 18, 28, 39, 45, 52, 55, 58, 78, 79,..."
2,[black sabbath],"[0, 1, 2, 13, 16, 17, 27, 30, 37, 41, 47, 51, ..."
3,[deep purple],"[3, 7, 17, 22, 40, 45, 59, 65, 85, 97, 103, 11..."
4,[george michael],"[13, 14, 26, 33, 42, 43, 45, 56, 62, 65, 83, 9..."
...,...,...
412,"[george michael, kanye west, kendrick lamar, t...","[674, 541, 709, 751]"
413,"[george michael, kanye west, kendrick lamar, t...","[674, 541, 709, 751]"
414,"[kanye west, kendrick lamar, megadeth, tyler, ...","[1082, 732, 709, 1950]"
415,"[kanye west, kendrick lamar, megadeth, tyler, ...","[1082, 732, 709, 1950]"


In [24]:
eclat_df

,Item,Transactions
0,[2pac],"[9, 14, 23, 35, 48, 57, 69, 70, 77, 83, 89, 95..."
1,[bjork],"[0, 6, 15, 18, 28, 39, 45, 52, 55, 58, 78, 79,..."
2,[black sabbath],"[0, 1, 2, 13, 16, 17, 27, 30, 37, 41, 47, 51, ..."
3,[deep purple],"[3, 7, 17, 22, 40, 45, 59, 65, 85, 97, 103, 11..."
4,[george michael],"[13, 14, 26, 33, 42, 43, 45, 56, 62, 65, 83, 9..."
...,...,...
412,"[george michael, kanye west, kendrick lamar, t...","[674, 541, 709, 751]"
413,"[george michael, kanye west, kendrick lamar, t...","[674, 541, 709, 751]"
414,"[kanye west, kendrick lamar, megadeth, tyler, ...","[1082, 732, 709, 1950]"
415,"[kanye west, kendrick lamar, megadeth, tyler, ...","[1082, 732, 709, 1950]"
